# Model

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Configuration
img_size = (224, 224)
batch_size = 32
data_dir = '/kaggle/input/audio-paper/train/train'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=8,        
    width_shift_range=0.08,  
    height_shift_range=0.08,
    zoom_range=0.1,          
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42,
    color_mode='rgb'
)

validation_data = val_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False,  
    color_mode='rgb'
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, DepthwiseConv2D
from tensorflow.keras.models import Model

def bc_resnet_block(x, filters, stride, name):
    """Basic Broadcasted Residual Block"""
    shortcut = x

    # Depthwise Convolution
    x = DepthwiseConv2D(kernel_size=3, strides=stride, padding='same', name=name+'_depthwise')(x)
    x = BatchNormalization(name=name+'_bn1')(x)
    x = ReLU(name=name+'_relu1')(x)

    # Pointwise Convolution
    x = Conv2D(filters, kernel_size=1, strides=1, padding='same', name=name+'_pointwise')(x)
    x = BatchNormalization(name=name+'_bn2')(x)

    # Broadcast skip connection if shapes don't match
    if shortcut.shape[-1] != filters or stride != 1:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same', name=name+'_skip_proj')(shortcut)
        shortcut = BatchNormalization(name=name+'_skip_bn')(shortcut)

    x = Add(name=name+'_add')([x, shortcut])
    x = ReLU(name=name+'_relu2')(x)
    return x

def bc_resnet_40(input_shape=(224, 224, 3), num_classes=10):
    inputs = Input(shape=input_shape)

    # Stem
    x = Conv2D(32, kernel_size=3, strides=2, padding='same', name='stem_conv')(inputs)
    x = BatchNormalization(name='stem_bn')(x)
    x = ReLU(name='stem_relu')(x)

    # 40 layers ≈ 20 blocks (each block has 2 convs technically)
    block_config = [2, 4, 6, 6, 2]  # Total blocks = 20
    filters = [32, 64, 128, 256, 512]

    for i, (n_blocks, f) in enumerate(zip(block_config, filters)):
        for j in range(n_blocks):
            stride = 2 if j == 0 and i != 0 else 1  # Downsample at the start of each stage (except first)
            x = bc_resnet_block(x, f, stride, name=f'stage{i+1}_block{j+1}')

    # Classification head
    x = GlobalAveragePooling2D(name='gap')(x)
    outputs = Dense(num_classes, activation='softmax', name='classifier')(x)

    model = Model(inputs, outputs, name='BCResNet40')
    return model

model = bc_resnet_40(input_shape=(224, 224, 3), num_classes=5)
model.summary()



I0000 00:00:1752121889.973740      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752121889.974469      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "BCResNet40"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_relu (ReLU)    │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_dept… │ (None, 112, 112,  │        320 │ stem_relu[0][0]   │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_bn1   │ (None, 112, 112,  │        128 │ stage1_block1_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_relu1 │ (None, 112, 112,  │          0 │ stage1_block1_bn… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_poin… │ (None, 112, 112,  │      1,056 │ stage1_block1_re… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_bn2   │ (None, 112, 112,  │        128 │ stage1_block1_po… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_add   │ (None, 112, 112,  │          0 │ stage1_block1_bn… │
│ (Add)               │ 32)               │            │ stem_relu[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block1_relu2 │ (None, 112, 112,  │          0 │ stage1_block1_ad… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_dept… │ (None, 112, 112,  │        320 │ stage1_block1_re… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_bn1   │ (None, 112, 112,  │        128 │ stage1_block2_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_relu1 │ (None, 112, 112,  │          0 │ stage1_block2_bn… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_poin… │ (None, 112, 112,  │      1,056 │ stage1_block2_re… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_bn2   │ (None, 112, 112,  │        128 │ stage1_block2_po… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stage1_block2_add   │ (None, 112, 112,  │          0 │ stage1_block2_bn

 Total params: 1,105,221 (4.22 MB)

 Trainable params: 1,089,605 (4.16 MB)

 Non-trainable params: 15,616 (61.00 KB)

In [ ]:
# For 20 epochs
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
#  For rest 80 epoch (Fine tuning)
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)